In [8]:
import sys
from datetime import date
import pandas as pd
import numpy as np
from jugaad_data.nse import stock_df

In [70]:
def get_ewm(DATA, i, long_term, alpha):
    if (i == 0):
        return DATA[0]
    
    prev_ewm = get_ewm(DATA, i-1, long_term, alpha)
    ewm = alpha*(DATA[i] - prev_ewm) + prev_ewm
    
    return ewm

In [95]:
def macd(path):
    df = pd.read_csv(path)
    stocks = 0
    cashflow = 0
    df['PriceChange'] = df['CLOSE'].diff()

    # Calculate average gain and average loss over a specified window (e.g., 14 days)
    window = 14
    df['Gain'] = np.where(df['PriceChange'] > 0, df['PriceChange'], 0)
    df['Loss'] = np.where(df['PriceChange'] < 0, -df['PriceChange'], 0)
    df['AverageGain'] = df['Gain'].rolling(window  = window, min_periods=1).mean()
    df['AverageLoss'] = df['Loss'].rolling(window  = window, min_periods=1).mean()

    # Calculate the Relative Strength (RS)
    df['RS'] = df['AverageGain'] / df['AverageLoss']

    # Calculate the RSI
    rsi = 100 - (100 / (1 + df['RS']))
    print(rs)
    # for i in range(247):
    #     if (i < 14):
    #         continue
    #     if (rsi[i] < 30 and stocks < 3):
    #         stocks += 1
    #         cashflow -= df['CLOSE'][i]
    #     elif  (rsi[i] > 70 and stocks > -3):
    #         stocks -= 1
    #         cashflow += df['CLOSE'][i]
    #     print(f"{df['DATE'][i]} {cashflow}")

In [96]:
# ------------------------------------ generation of data frame --------------------------------------------
def generate_dataframe(symbol, today, lastday, train_data):
    # formatting the date
    to_day = int(today[:2])
    to_month = int(today[3:5])
    to_year = int(today[6:10])
    
    la_day = int(lastday[:2])
    la_month = int(lastday[3:5])
    la_year = int(lastday[6:10])
    
    df = pd.DataFrame(stock_df(symbol=symbol, from_date=date(to_year, to_month, to_day), to_date=date(la_year, la_month, la_day), series="EQ"))
    df = df[[ "DATE", "CLOSE", "HIGH", "LOW", "PREV. CLOSE", "VWAP", "NO OF TRADES", "OPEN"]]
    df = df.iloc[::-1]
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%m-%Y')
    df['DATE'] = df['DATE'].dt.strftime('%d/%m/%Y')
    if (train_data == "0"):
        df.to_csv("Stocks/"+symbol+".csv", index=False)
    else:
        df.to_csv("Stocks/"+symbol+"_train.csv", index = False)

# ----------------------------------------------- pickle --------------------------------------------------------
def write_pickle(DATA, symbol):
    pd.to_pickle(DATA, symbol + ".pkl")

# -------------------------------------------------------- MAIN -----------------------------------------------------------------------
def main():
    argument = "SBIN"
    lastday = "01/01/2024"
    today = "01/01/2023"
    train_data = "0"

    generate_dataframe(argument, today, lastday, train_data)
    #write_pickle(DATA, "Stocks/" + argument)
    macd("Stocks/SBIN.csv")

if __name__ == "__main__":
    main()

0             NaN
1      100.000000
2        2.702703
3        2.666667
4        1.673640
          ...    
242     67.447307
243     70.950888
244     70.828961
245     63.649706
246     63.180934
Name: RS, Length: 247, dtype: float64
